In [1]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

sk


In [2]:
# %pip install langchain_community faiss-cpu
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from pprint import pprint

In [9]:

# 1. Load Data
loader = TextLoader("data/taxinfo.txt", encoding="utf-8")
documents = loader.load()

# 2️. Text Split
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = splitter.split_documents(documents)
#print(split_docs)

# 3️. Indexing (벡터 저장)
vectorstore = FAISS.from_documents(split_docs, OpenAIEmbeddings())
# 로컬 파일로 저장
vectorstore.save_local("faiss_index")

# 4️. Retrieval (유사 문서 검색)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
# **질문(쿼리)**에 대해 유사한 문서를 검색하는 역할
retrieved_docs = retriever.invoke("소득세법에서 비과세소득에 해당하는 소득은 무엇인가요?")
#print(retrieved_docs)

# 5️. Generation (LLM 응답 생성)
llm = ChatOpenAI(model="gpt-4o")
context = "\n\n".join([doc.page_content for doc in retrieved_docs])
#print(context)

In [10]:

response_context = llm.invoke(f"소득세법에서 비과세소득에 해당하는 소득은 무엇인가요? 관련 정보: {context}")
print('context 적용한 결과')
pprint(response_context.content)


context 적용한 결과
('소득세법 제12조에 따르면 여러 종류의 소득이 비과세소득으로 분류되어 소득세가 부과되지 않습니다. 주요 비과세소득 항목은 다음과 '
 '같습니다:\n'
 '\n'
 '1. **공익신탁의 이익**: 「공익신탁법」에 따라 발생하는 공익신탁의 이익은 비과세됩니다.\n'
 '\n'
 '2. **일부 사업소득**:\n'
 '   - **농업 소득**: 논과 밭을 작물 생산에 이용하여 발생하는 소득은 비과세됩니다.\n'
 '   - **주택임대소득**: 1개의 주택을 소유한 자의 주택임대소득 중 일부는 비과세됩니다. 단, 제99조에 따른 기준시가가 12억원을 '
 '초과하는 주택 및 국외에 소재하는 주택의 임대소득은 과세 대상입니다. 또한, 해당 과세기간에 총수입금액의 합계액이 2천만원 이하인 자의 '
 '주택임대소득(2018년 12월 31일 이전까지 발생하는 소득)은 비과세됩니다.\n'
 '   - **농어가부업소득**: 대통령령으로 정하는 농어가부업소득이 비과세됩니다.\n'
 '   - **전통주의 제조 소득**: 대통령령으로 정하는 전통주의 제조에서 발생하는 소득은 비과세됩니다.\n'
 '   - **임목의 벌채 또는 양도 소득**: 조림기간 5년 이상인 임지의 임목 벌채 또는 양도로 발생하는 연 600만원 이하의 소득은 '
 '비과세됩니다. 조림기간 및 세액 계산에 관한 사항은 대통령령에 따릅니다.\n'
 '   - **작물재배업 소득**: 대통령령으로 정하는 작물재배업에서 발생하는 소득은 비과세됩니다.\n'
 '\n'
 '이 항목들은 여러 법률 개정을 거쳐 현재의 규정에 이르게 되었으며, 구체적인 사항은 관련 대통령령에 따릅니다. 이를 통해 특정 조건을 '
 '만족하면 일부 소득은 비과세 혜택을 받을 수 있습니다.')


In [11]:

response = llm.invoke(f"소득세법에서 비과세소득에 해당하는 소득은 무엇인가요?")
print('context 적용하지 않은 결과')
pprint(response.content)

context 적용하지 않은 결과
('소득세법에서 비과세소득은 과세 대상에서 제외되어 세금을 부과하지 않는 소득을 의미합니다. 한국 소득세법에 따라 비과세소득으로 인정되는 '
 '항목에는 다음과 같은 것들이 포함될 수 있습니다:\n'
 '\n'
 '1. **국민연금 및 기타 공적연금의 일부**: 일정 금액 이하의 연금소득은 비과세로 처리될 수 있습니다.\n'
 '   \n'
 '2. **고용보험 및 산업재해보상보험에 의한 급여**: 예를 들어 실업급여나 산업재해로 인한 보상금 등이 해당됩니다.\n'
 '\n'
 '3. **장애인 연금**: 장애인 복지를 위한 일부 연금은 비과세될 수 있습니다.\n'
 '\n'
 '4. **학자금 및 장학금**: 교육 지원을 목적으로 하는 장학금 중 일정 조건을 충족하는 경우 비과세로 인정됩니다.\n'
 '\n'
 '5. **일정한 저축성 보험의 환급금**: 일정 요건을 충족하는 저축성 보험의 환급금이 비과세될 수 있습니다.\n'
 '\n'
 '6. **국가 및 지방자치단체가 지급하는 보조금 중 일부**: 특정한 요건 하에서 제공되는 보조금은 비과세 소득으로 인정됩니다.\n'
 '\n'
 '7. **기타 법령에 의해 비과세로 규정된 소득**: 법령에 따라 추가적으로 정해진 다양한 비과세 소득 항목들이 있을 수 있습니다.\n'
 '\n'
 '자세한 사항은 법령 개정에 따라 변경될 수 있으므로, 최신 정보는 한국 국세청이나 관련 법령 문서를 통해 확인하는 것이 좋습니다.')


### 1차 개선

In [12]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from pprint import pprint

# 1. 데이터 로드 (기존과 동일)
loader = TextLoader("data/taxinfo.txt", encoding="utf-8")
documents = loader.load()

# 2. 텍스트 분할 개선
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # 크기 증가
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", ""],  # 자연스러운 분할을 위한 구분자
    length_function=len,
    is_separator_regex=False,
)
split_docs = splitter.split_documents(documents)

# 3. 인덱싱 (벡터 저장)
vectorstore = FAISS.from_documents(split_docs, OpenAIEmbeddings())
vectorstore.save_local("faiss_index")

# 4. 검색 개선
retriever = vectorstore.as_retriever(
    search_type="mmr",  # 최대 다양성 검색
    search_kwargs={"k": 5, "fetch_k": 10}  # 더 많은 결과 검색
)

# 5. 프롬프트 엔지니어링
def generate_prompt(query, context):
    return f"""다음은 소득세법 비과세소득 관련 조항입니다. 문맥을 고려하여 질문에 답변하세요.

[관련 조항]
{context}

[질문]
{query}

[답변 요구사항]
- 비과세소득 유형을 계층적으로 구분하여 설명
- 각 항목별 구체적인 조건 명시
- 법조문의 항, 호, 목 번호를 포함
- 500자 이내로 간결하게 요약"""

# 검색 및 응답 생성
query = "소득세법에서 비과세소득에 해당하는 소득은 무엇인가요?"
retrieved_docs = retriever.invoke(query)
context = "\n\n".join([doc.page_content for doc in retrieved_docs])

llm = ChatOpenAI(model="gpt-4o", temperature=0.3)  # 창의성 낮춤
response = llm.invoke(generate_prompt(query, context))

print('개선된 결과:')
pprint(response.content)

개선된 결과:
('소득세법 제12조에 따른 비과세소득은 다음과 같이 구분됩니다:\n'
 '\n'
 '1. **공익신탁 이익**: 공익신탁법에 따른 이익(제12조 1호).\n'
 '\n'
 '2. **사업소득**: \n'
 '   - 작물 생산을 위한 논밭 임대 소득(제12조 2호 가목).\n'
 '   - 1주택 소유자의 일정 조건 주택임대소득(제12조 2호 나목).\n'
 '   - 농어가부업소득, 전통주 제조 소득 등 대통령령으로 정하는 소득(제12조 2호 다~사목).\n'
 '\n'
 '3. **근로소득 및 퇴직소득**:\n'
 '   - 국외근로 급여, 특정 보험료, 연장근로 수당 등(제12조 3호 거~어목).\n'
 '\n'
 '4. **기타소득**:\n'
 '   - 보훈급여금, 상금, 직무발명보상금 등(제12조 5호 가~자목).\n'
 '\n'
 '각 항목은 대통령령 등에서 정한 구체적인 조건을 충족해야 비과세 혜택을 받을 수 있습니다.')


### 2차 개선

In [14]:
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from pprint import pprint

# 1. Load Data
loader = TextLoader("data/taxinfo.txt", encoding="utf-8")
documents = loader.load()

print("=== 원본 문서 길이 ===")
print(f"전체 문서 길이: {len(documents[0].page_content)} 글자")

# 2. Text Split 개선
splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,  # 500 → 800 (법령 조항이 길어서)
    chunk_overlap=150,  # 50 → 150 (맥락 보존 강화)
    separators=["\n\n", "\n", ". ", " ", ""]  # 법령 구조에 맞는 분리자
)
split_docs = splitter.split_documents(documents)

print(f"분할된 문서 수: {len(split_docs)}개")
print("=== 분할 예시 ===")
for i, doc in enumerate(split_docs[:3]):
    print(f"Chunk {i+1} ({len(doc.page_content)}글자): {doc.page_content[:100]}...")

# 3. Indexing
vectorstore = FAISS.from_documents(split_docs, OpenAIEmbeddings())
vectorstore.save_local("faiss_index")

# 4. Retrieval 개선
retriever = vectorstore.as_retriever(
    search_type="similarity", 
    search_kwargs={"k": 6}  # 2 → 6으로 증가
)

query = "소득세법에서 비과세소득에 해당하는 소득은 무엇인가요?"
retrieved_docs = retriever.invoke(query)

print(f"\n=== 검색된 문서 ({len(retrieved_docs)}개) ===")
for i, doc in enumerate(retrieved_docs):
    print(f"문서 {i+1}: {doc.page_content[:200]}...")
    print("---")

# 5. Generation - 개선된 프롬프트
llm = ChatOpenAI(model="gpt-4o", temperature=0)
context = "\n\n".join([f"[문서 {i+1}]\n{doc.page_content}" for i, doc in enumerate(retrieved_docs)])

# 개선된 프롬프트 - 더 구체적인 지시사항
improved_prompt = f"""
당신은 세무 전문가입니다. 아래 소득세법 제12조 조항을 바탕으로 질문에 답변해주세요.

질문: {query}

법령 조항:
{context}

다음 형식으로 답변해주세요:
1. 비과세소득의 정의
2. 주요 비과세소득 항목들을 다음과 같이 분류:
   - 사업소득 관련
   - 근로소득/퇴직소득 관련  
   - 연금소득 관련
   - 기타소득 관련
3. 각 항목별 구체적인 조건이나 한도액 명시

답변은 법조문을 인용하면서 구체적으로 작성해주세요.
"""

# 비교용 - 기존 방식
simple_prompt = f"소득세법에서 비과세소득에 해당하는 소득은 무엇인가요? 관련 정보: {context}"

print("\n=== 개선된 프롬프트로 답변 ===")
response_improved = llm.invoke(improved_prompt)
pprint(response_improved.content)

print("\n" + "="*50)
print("=== 기존 프롬프트로 답변 ===")
response_simple = llm.invoke(simple_prompt)
pprint(response_simple.content)

# 추가 개선: 다른 검색 방식 시도
print("\n" + "="*50)
print("=== 검색 방식 개선 테스트 ===")


=== 원본 문서 길이 ===
전체 문서 길이: 4969 글자
분할된 문서 수: 8개
=== 분할 예시 ===
Chunk 1 (738글자): 제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2....
Chunk 2 (636글자): 다. 대통령령으로 정하는 농어가부업소득
    라. 대통령령으로 정하는 전통주의 제조에서 발생하는 소득
    마. 조림기간 5년 이상인 임지(林地)의 임목(林木)의 벌채 또는 양...
Chunk 3 (792글자): 라. 「근로기준법」 또는 「선원법」에 따라 근로자ㆍ선원 및 그 유족이 받는 요양보상금, 휴업보상금, 상병보상금(傷病補償金), 일시보상금, 장해보상금, 유족보상금, 행방불명보상금, ...

=== 검색된 문서 (6개) ===
문서 1: 제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 2...
---
문서 2: 2) 대학의 교직원 또는 대학과 고용관계가 있는 학생이 소속 대학에 설치된 「산업교육진흥 및 산학연협력촉진에 관한 법률」 제25조에 따른 산학협력단(이하 이 조에서 “산학협력단”이라 한다)으로부터 같은 법 제32조제1항제4호에 따라 받는 보상금
    저. 대통령령으로 정하는 복리후생적 성질의 급여
4. 연금소득 중 다음 각 목의 어느 하나에 해당하는 소득...
---
문서 3: 나. 「국가보안법」에 따라 받는 상금과 보로금
    다. 「상훈법」에 따른 훈장과 관련하여 받는 부상(副賞)이나 그 밖에 대통령령으로 정하는 상금과 부상
    라. 종업원등 또는 대학의 교직원이 퇴직한 후에 사용

In [15]:

# MMR(Maximum Marginal Relevance) 검색 - 다양성 확보
retriever_mmr = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 6, "fetch_k": 20}
)
retrieved_docs_mmr = retriever_mmr.invoke(query)
context_mmr = "\n\n".join([f"[문서 {i+1}]\n{doc.page_content}" for i, doc in enumerate(retrieved_docs_mmr)])

response_mmr = llm.invoke(f"""
{query}

법령 조항 (MMR 검색):
{context_mmr}

위 법령을 바탕으로 비과세소득 항목들을 체계적으로 정리해주세요.
""")

print("=== MMR 검색 결과 ===")
pprint(response_mmr.content)

=== MMR 검색 결과 ===
('소득세법 제12조에 따른 비과세소득 항목은 다음과 같이 체계적으로 정리할 수 있습니다.\n'
 '\n'
 '1. **공익신탁의 이익**\n'
 '   - 「공익신탁법」에 따른 공익신탁의 이익\n'
 '\n'
 '2. **사업소득**\n'
 '   - 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n'
 '   - 1개의 주택을 소유하는 자의 주택임대소득 (단, 기준시가가 12억원을 초과하는 주택 및 국외에 소재하는 주택의 임대소득은 '
 '제외)\n'
 '   - 대통령령으로 정하는 농어가부업소득\n'
 '   - 대통령령으로 정하는 전통주의 제조에서 발생하는 소득\n'
 '   - 조림기간 5년 이상인 임지의 임목의 벌채 또는 양도로 발생하는 소득으로서 연 600만원 이하의 금액\n'
 '   - 대통령령으로 정하는 작물재배업에서 발생하는 소득\n'
 '   - 대통령령으로 정하는 어로어업 또는 양식어업에서 발생하는 소득\n'
 '\n'
 '3. **근로소득과 퇴직소득**\n'
 '   - 대통령령으로 정하는 복무 중인 병이 받는 급여\n'
 '   - 법률에 따라 동원된 사람이 그 동원 직장에서 받는 급여\n'
 '   - 「산업재해보상보험법」에 따라 수급권자가 받는 각종 보상금\n'
 '   - 「근로기준법」 또는 「선원법」에 따라 근로자ㆍ선원 및 그 유족이 받는 각종 보상금\n'
 '   - 「고용보험법」에 따라 받는 실업급여 등\n'
 '   - 「국민연금법」에 따라 받는 반환일시금 및 사망일시금\n'
 '   - 「공무원연금법」 등 관련 법령에 따라 받는 각종 보상금 및 급여\n'
 '   - 대통령령으로 정하는 학자금\n'
 '\n'
 '4. **연금소득**\n'
 '   - 「국민연금법」, 「공무원연금법」 등 공적연금 관련법에 따라 받는 각종 연금\n'
 '   - 「산업재해보상보험법」에 따라 받는 각종 연금\n'
 '   - 「국군포로의 송환 및 대우 등에 관한 법률」에 따른 연금\n'
 '\n'
 '